In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
import keras
from keras.models import Sequential 
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import GlobalAveragePooling2D
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.utils.vis_utils import plot_model

In [3]:
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In [4]:
classifier = Sequential()

2022-04-21 03:36:21.960041: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
classifier.add(Conv2D(32, (3, 3), strides=1, padding='same', activation='relu', input_shape=[64, 64, 3]))
classifier.add(BatchNormalization())
classifier.add(MaxPool2D((2,2), strides=2, padding='same'))

In [6]:
classifier.add(Conv2D(64, (3,3), strides=1, padding='same', activation='relu'))
classifier.add(Dropout(0.2))
classifier.add(BatchNormalization())
classifier.add(MaxPool2D((2,2), strides=2, padding='same'))

In [7]:
classifier.add(Conv2D(64, (3,3), strides=1, padding='same', activation='relu'))
classifier.add(BatchNormalization())
classifier.add(MaxPool2D((2, 2), strides = 2, padding = 'same'))

In [8]:
classifier.add(Conv2D(128, (3, 3), strides = 1, padding = 'same', activation = 'relu'))
classifier.add(Dropout(0.2))
classifier.add(BatchNormalization())
classifier.add(MaxPool2D((2, 2), strides = 2, padding = 'same'))
classifier.add(Flatten())
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.2))
#classifier.add(GlobalAveragePooling2D())

In [9]:
classifier.add(Dense(1, activation='sigmoid'))

In [10]:
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0

In [11]:
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    shear_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True
                )

In [12]:
test_datagen = ImageDataGenerator(rescale=1./255)

In [13]:
training_set = train_datagen.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/train',
                                                    target_size=(64,64),
                                                    batch_size=32,
                                                    class_mode='binary'
                                                )

Found 5216 images belonging to 2 classes.


In [14]:
test_set = test_datagen.flow_from_directory('../input/chest-xray-pneumonia/chest_xray/test',
                                                    target_size=(64,64),
                                                    batch_size=32,
                                                    class_mode='binary'
                                                )

Found 624 images belonging to 2 classes.


In [15]:
classifier.fit(x = training_set, validation_data = test_set, epochs = 10)

2022-04-21 03:36:26.950477: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
163/163 [==============================] - 129s 786ms/step - loss: 0.3055 - accuracy: 0.8903 - val_loss: 4.8557 - val_accuracy: 0.6250
Epoch 2/10
163/163 [==============================] - 92s 566ms/step - loss: 0.1716 - accuracy: 0.9379 - val_loss: 3.9487 - val_accuracy: 0.6250
Epoch 3/10
163/163 [==============================] - 91s 558ms/step - loss: 0.1557 - accuracy: 0.9415 - val_loss: 1.6932 - val_accuracy: 0.6250
Epoch 4/10
163/163 [==============================] - 92s 564ms/step - loss: 0.1502 - accuracy: 0.9452 - val_loss: 3.0102 - val_accuracy: 0.6250
Epoch 5/10
163/163 [==============================] - 91s 556ms/step - loss: 0.1345 - accuracy: 0.9496 - val_loss: 1.2359 - val_accuracy: 0.6106
Epoch 6/10
163/163 [==============================] - 90s 550ms/step - loss: 0.1285 - accuracy: 0.9559 - val_loss: 1.0883 - val_accuracy: 0.6667
Epoch 7/10
163/163 [==============================] - 90s 553ms/step - loss: 0.1202 - accuracy: 0.9563 - val_loss: 0.4825 - val_a

In [16]:
from keras.preprocessing import image

In [17]:
test_image = image.load_img('../input/chest-xray-pneumonia/chest_xray/train/PNEUMONIA/person1000_bacteria_2931.jpeg', target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0) 
result = classifier.predict(test_image)
training_set.class_indices
if result[0][0] == 1:
    prediction = 'Pneumonia'
else:
    prediction = 'Normal'

In [18]:
print(prediction)

Pneumonia
